In [ ]:
#No Renewables, LP without Binary Variables, with A/S, Separate Objective Functions
import pandas as pd
import numpy as np
import math
import sys
url = None
key = None
from docplex.mp.model import Model
multi_batts_data = pd.read_csv('Multi_Batts_LP_Data.csv')
batt = multi_batts_data.values

#Set Parameters
num_batt = 463
batt_name = list(multi_batts_data.columns[7:(num_batt+7)])
num_hr = len(batt); num_day = math.ceil(num_hr/24)
batt_size = batt[:,1].tolist(); batt_cap = batt[:,2].tolist(); start_inv = batt[:,3].tolist()
rd_pr = batt[:,4].tolist(); ru_pr = batt[:,5].tolist(); sp_pr = batt[:,6].tolist()
header = [""]; gen_h = {}; regdn_h = {}; regup_h = {}; spin_h = {}; soc_h = {}
for k in range(0,num_batt):
    gen_h[k] = "Gen{0}".format(k+1)
    regdn_h[k] = "RegDown{0}".format(k+1)
    regup_h[k] = "RegUp{0}".format(k+1)
    spin_h[k] = "Spin{0}".format(k+1)
    soc_h[k] = "SOC{0}".format(k+1)
eff_c=0.85; krd=0.1; kru=0.1; ksp=0.1
gen_val = []; rd_val = []; ru_val = []; sp_val = []; soc_val = []

#Perform LP Optimization
for k in range(0,num_batt):
    m = Model(name='Batt_Sched')
    lmps = batt[:,7+k].tolist()
    charge = m.continuous_var_list(num_hr)
    disch = m.continuous_var_list(num_hr)
    regdn = m.continuous_var_list(num_hr)
    regup = m.continuous_var_list(num_hr)
    spin = m.continuous_var_list(num_hr)
    soc = m.continuous_var_list(num_hr, ub = batt_cap[k])
    m.add_constraints(charge[i] + disch[i] + regdn[i] + regup[i] + spin[i] <= batt_size[k] for i in range(num_hr))
    m.add_constraint(eff_c*charge[0]-disch[0]+eff_c*krd*regdn[0]-kru*regup[0]-ksp*spin[0]-soc[0] == -start_inv[k])
    m.add_constraints(eff_c*charge[i]-disch[i]+eff_c*krd*regdn[i]-kru*regup[i]-ksp*spin[i]+soc[i-1]-soc[i] == 0 for i in range(1,num_hr))
    m.add_constraints(soc[i-1] + eff_c*(regdn[i] + charge[i]) <= batt_cap[k] for i in range(1,num_hr))
    m.add_constraints(soc[i-1] - regup[i] - spin[i] - disch[i] >= 0 for i in range(1,num_hr))
    m.add_constraints(m.sum(regdn[24*j+i]+regup[24*j+i]+spin[24*j+i]  for i in range(24)) <= 2.4*batt_size[k] for j in range(num_day))
    m.add_constraints(m.sum(eff_c*charge[24*j+i]+eff_c*krd*regdn[24*j+i]  for i in range(24)) <= batt_cap[k] for j in range(num_day))
    m.maximize(m.sum(-lmps[i]*charge[i]+lmps[i]*disch[i]+rd_pr[i]*regdn[i]+ru_pr[i]*regup[i]+sp_pr[i]*spin[i] for i in range(num_hr)))
    m.solve()
    gen_val += [disch[i].solution_value - charge[i].solution_value for i in range(num_hr)]
    rd_val += [regdn[i].solution_value for i in range(num_hr)]
    ru_val += [regup[i].solution_value for i in range(num_hr)]
    sp_val += [spin[i].solution_value for i in range(num_hr)]
    soc_val += [soc[i].solution_value for i in range(num_hr)]
    header += [batt_name[k],"","Profit =",m.objective_value,""]
    del m
    
#Produce Results    
sol_values = pd.DataFrame(data={"Date/Time": multi_batts_data["Date/Time"]})
for k in range(0,num_batt):
    sol_values[gen_h[k]] = gen_val[k*num_hr:(k+1)*num_hr]
    sol_values[regdn_h[k]] = rd_val[k*num_hr:(k+1)*num_hr]
    sol_values[regup_h[k]] = ru_val[k*num_hr:(k+1)*num_hr]
    sol_values[spin_h[k]] = sp_val[k*num_hr:(k+1)*num_hr]
    sol_values[soc_h[k]] = soc_val[k*num_hr:(k+1)*num_hr]
sol_values.columns = pd.MultiIndex.from_tuples(zip(header,sol_values.columns))
sol_values.to_csv("./multi_batt_result.csv", sep=',',index=False)

In [ ]:
#Renewables, LP without Binary Variables, with A/S, Separate Objective Functions
import pandas as pd
import numpy as np
import math
import sys
url = None
key = None
from docplex.mp.model import Model
multi_batts_data = pd.read_csv('Multi_Batts_LP_Data_Renewables.csv')
batt = multi_batts_data.values
solar_profiles = pd.read_csv('solar_profiles.csv')
profiles = solar_profiles.values

#Set Parameters
num_batt = 882
batt_name = list(multi_batts_data.columns[9:(num_batt+9)])
num_hr = len(batt); num_day = math.ceil(num_hr/24)
batt_size = batt[:,1].tolist(); batt_cap = batt[:,2].tolist(); start_inv = batt[:,3].tolist()
rd_pr = batt[:,6].tolist(); ru_pr = batt[:,7].tolist(); sp_pr = batt[:,8].tolist(); 
min_re = 0.01*batt[0:num_batt,4]; counties = batt[0:num_batt,5].tolist(); re_size = profiles[0:num_batt,1].tolist()
header = [""]; gen_h = {}; re_h = {}; regdn_h = {}; regup_h = {}; spin_h = {}; soc_h = {}
for k in range(0,num_batt):
    gen_h[k] = "Gen{0}".format(k+1)
    re_h[k] = "ReCharge{0}".format(k+1)
    regdn_h[k] = "RegDown{0}".format(k+1)
    regup_h[k] = "RegUp{0}".format(k+1)
    spin_h[k] = "Spin{0}".format(k+1)
    soc_h[k] = "SOC{0}".format(k+1)
eff_c=0.85; eff_re=0.85; krd=0.1; kru=0.1; ksp=0.1
gen_val = []; renew_val = []; rd_val = []; ru_val = []; sp_val = []; soc_val = []

#Perform LP optimization
for k in range(0,num_batt):
    m = Model(name='Batt_Sched')
    lmps = batt[:,9+k].tolist()
    renewable_prof = solar_profiles["{0}".format(counties[k])].values
    renewable_prof_mw = [i*re_size[k]*0.01 for i in renewable_prof]
    charge = m.continuous_var_list(num_hr)
    disch = m.continuous_var_list(num_hr)
    renew = m.continuous_var_list(num_hr, ub=renewable_prof_mw)
    regdn = m.continuous_var_list(num_hr)
    regup = m.continuous_var_list(num_hr)
    spin = m.continuous_var_list(num_hr)
    soc = m.continuous_var_list(num_hr, ub = batt_cap[k])
    m.add_constraints(charge[i] + renew[i] + disch[i] + regdn[i] + regup[i] + spin[i] <= batt_size[k] for i in range(num_hr))
    m.add_constraints(soc[i-1] + eff_c*(regdn[i] + charge[i]) + eff_re*renew[i] <= batt_cap[k] for i in range(1,num_hr))
    m.add_constraints(soc[i-1] - regup[i] - spin[i] - disch[i] >= 0 for i in range(1,num_hr))
    m.add_constraint(eff_c*charge[0]+eff_re*renew[0]-disch[0]+eff_c*krd*regdn[0]-kru*regup[0]-ksp*spin[0]-soc[0] == -start_inv[k])
    m.add_constraints(eff_c*charge[i]+eff_re*renew[i]-disch[i]+eff_c*krd*regdn[i]-kru*regup[i]-ksp*spin[i]+soc[i-1]-soc[i] == 0 for i in range(1,num_hr))
    m.add_constraints(m.sum(eff_c*charge[24*j+i]+eff_re*renew[24*j+i]+eff_c*krd*regdn[24*j+i]  for i in range(24)) <= batt_cap[k] for j in range(num_day))
    m.add_constraints(m.sum(regdn[24*j+i]+regup[24*j+i]+spin[24*j+i]  for i in range(24)) <= 2.4*batt_size[k] for j in range(num_day))
    m.add_constraints(m.sum(min_re[k]*(charge[24*j+i]+krd*regdn[24*j+i])-(1-min_re[k])*renew[24*j+i] for i in range(24))<= 0 for j in range(num_day))    #Min Charge from Renewables
    m.maximize(m.sum(-lmps[i]*(charge[i]+renew[i])+lmps[i]*disch[i]+rd_pr[i]*regdn[i]+ru_pr[i]*regup[i]+sp_pr[i]*spin[i] for i in range(num_hr)))
    m.solve()
    gen_val += [disch[i].solution_value - charge[i].solution_value for i in range(num_hr)]
    renew_val += [renew[i].solution_value for i in range(num_hr)]
    rd_val += [regdn[i].solution_value for i in range(num_hr)]
    ru_val += [regup[i].solution_value for i in range(num_hr)]
    sp_val += [spin[i].solution_value for i in range(num_hr)]
    soc_val += [soc[i].solution_value for i in range(num_hr)]
    renew_income = sum([lmps[i]*renewable_prof_mw[i] for i in range(num_hr)])
    header += [batt_name[k],"Profit =",m.objective_value,"Solar Income =",renew_income,""]
    del m

#Save Results
sol_values = pd.DataFrame(data={"Date/Time": multi_batts_data["Date/Time"]})
for k in range(0,num_batt):
    sol_values[gen_h[k]] = gen_val[k*num_hr:(k+1)*num_hr]
    sol_values[re_h[k]] = renew_val[k*num_hr:(k+1)*num_hr]
    sol_values[regdn_h[k]] = rd_val[k*num_hr:(k+1)*num_hr]
    sol_values[regup_h[k]] = ru_val[k*num_hr:(k+1)*num_hr]
    sol_values[spin_h[k]] = sp_val[k*num_hr:(k+1)*num_hr]
    sol_values[soc_h[k]] = soc_val[k*num_hr:(k+1)*num_hr]
sol_values.columns = pd.MultiIndex.from_tuples(zip(header,sol_values.columns))
sol_values.to_csv("./multi_batt_result.csv", sep=',',index=False)

In [ ]:
#No Renewables, LP without Binary Variables, with Calculated A/S
import pandas as pd
import numpy as np
import math
import sys
url = None
key = None
from docplex.mp.model import Model
multi_batts_data = pd.read_csv('Multi_Batts_LP_Data.csv')
batt = multi_batts_data.values
AS_Data = pd.read_csv('AS Data.csv')

#Set Parameters
num_batt = 882
batt_name = list(multi_batts_data.columns[7:(num_batt+7)])
num_hr = len(batt); num_day = math.ceil(num_hr/24)
batt_size = batt[:,1].tolist(); batt_cap = batt[:,2].tolist(); start_inv = batt[:,3].tolist()
#rd_pr = batt[:,4].tolist(); ru_pr = batt[:,5].tolist(); sp_pr = batt[:,6].tolist()
header = [""]; gen_h = {}; regdn_h = {}; regup_h = {}; spin_h = {}; soc_h = {}
for k in range(0,num_batt):
    gen_h[k] = "Gen{0}".format(k+1)
    regdn_h[k] = "RegDown{0}".format(k+1)
    regup_h[k] = "RegUp{0}".format(k+1)
    spin_h[k] = "Spin{0}".format(k+1)
    soc_h[k] = "SOC{0}".format(k+1)
eff_c=0.85; krd=0.1; kru=0.1; ksp=0.1
gen_val = []; rd_val = []; ru_val = []; sp_val = []; soc_val = []

ihr = []; avg_lmp = AS_Data["ERCOT LMP"].tolist(); fuel_price = AS_Data["Fuel Price"][0:12].tolist()
month = [int(AS_Data["Date/Time"][i].split('/')[0]) for i in range(num_hr)]
rd_pr = []; ru_pr = []; sp_pr = []
for i in range(0,num_hr):
    ihr += [avg_lmp[i]/fuel_price[month[i]-1]]
    if(month[i]==1 or month[i]==2 or month[i]==12): season = "Winter"
    elif(month[i]==3 or month[i]==4 or month[i]==5): season = "Spring"
    elif(month[i]==6 or month[i]==7 or month[i]==8):season = "Summer"
    else: season = "Fall"
    as_ind = AS_Data.loc[(AS_Data["POP"]==AS_Data["Peak/Off-Peak"][i]) & (AS_Data["AS Type"]=="Regulation Down") & (AS_Data["Season"]==season)].index[0]    
    if(ihr[i]<AS_Data["Break point 1"][as_ind]): rd_pr += [AS_Data["Intercept 1"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 2"][as_ind] or AS_Data["Break point 2"][as_ind]==-1): rd_pr += [AS_Data["Intercept 2"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 3"][as_ind] or AS_Data["Break point 3"][as_ind]==-1): rd_pr += [AS_Data["Intercept 3"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 4"][as_ind] or AS_Data["Break point 4"][as_ind]==-1): rd_pr += [AS_Data["Intercept 4"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 5"][as_ind] or AS_Data["Break point 5"][as_ind]==-1): rd_pr += [AS_Data["Intercept 5"][as_ind]*ihr[i]]
    else: rd_pr += [AS_Data["Intercept 6"][as_ind]*ihr[i]]
    as_ind = AS_Data.loc[(AS_Data["POP"]==AS_Data["Peak/Off-Peak"][i]) & (AS_Data["AS Type"]=="Regulation Up") & (AS_Data["Season"]==season)].index[0]
    if(ihr[i]<AS_Data["Break point 1"][as_ind]): ru_pr += [AS_Data["Intercept 1"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 2"][as_ind] or AS_Data["Break point 2"][as_ind]==-1): ru_pr += [AS_Data["Intercept 2"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 3"][as_ind] or AS_Data["Break point 3"][as_ind]==-1): ru_pr += [AS_Data["Intercept 3"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 4"][as_ind] or AS_Data["Break point 4"][as_ind]==-1): ru_pr += [AS_Data["Intercept 4"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 5"][as_ind] or AS_Data["Break point 5"][as_ind]==-1): ru_pr += [AS_Data["Intercept 5"][as_ind]*ihr[i]]
    else: ru_pr += [AS_Data["Intercept 6"][as_ind]*ihr[i]]
    as_ind = AS_Data.loc[(AS_Data["POP"]==AS_Data["Peak/Off-Peak"][i]) & (AS_Data["AS Type"]=="Responsive Reserve Service") & (AS_Data["Season"]==season)].index[0]
    if(ihr[i]<AS_Data["Break point 1"][as_ind]): sp_pr += [AS_Data["Intercept 1"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 2"][as_ind] or AS_Data["Break point 2"][as_ind]==-1): sp_pr += [AS_Data["Intercept 2"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 3"][as_ind] or AS_Data["Break point 3"][as_ind]==-1): sp_pr += [AS_Data["Intercept 3"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 4"][as_ind] or AS_Data["Break point 4"][as_ind]==-1): sp_pr += [AS_Data["Intercept 4"][as_ind]*ihr[i]]
    elif(ihr[i]<AS_Data["Break point 5"][as_ind] or AS_Data["Break point 5"][as_ind]==-1): sp_pr += [AS_Data["Intercept 5"][as_ind]*ihr[i]]
    else: sp_pr += [AS_Data["Intercept 6"][as_ind]*ihr[i]]

#Perform LP Optimization
for k in range(0,num_batt):
    m = Model(name='Batt_Sched')
    lmps = batt[:,7+k].tolist()
    charge = m.continuous_var_list(num_hr)
    disch = m.continuous_var_list(num_hr)
    regdn = m.continuous_var_list(num_hr)
    regup = m.continuous_var_list(num_hr)
    spin = m.continuous_var_list(num_hr)
    soc = m.continuous_var_list(num_hr, ub = batt_cap[k])
    m.add_constraints(charge[i] + disch[i] + regdn[i] + regup[i] + spin[i] <= batt_size[k] for i in range(num_hr))
    m.add_constraint(eff_c*charge[0]-disch[0]+eff_c*krd*regdn[0]-kru*regup[0]-ksp*spin[0]-soc[0] == -start_inv[k])
    m.add_constraints(eff_c*charge[i]-disch[i]+eff_c*krd*regdn[i]-kru*regup[i]-ksp*spin[i]+soc[i-1]-soc[i] == 0 for i in range(1,num_hr))
    m.add_constraints(soc[i-1] + eff_c*(regdn[i] + charge[i]) <= batt_cap[k] for i in range(1,num_hr))
    m.add_constraints(soc[i-1] - regup[i] - spin[i] - disch[i] >= 0 for i in range(1,num_hr))
    m.add_constraints(m.sum(regdn[24*j+i]+regup[24*j+i]+spin[24*j+i]  for i in range(24)) <= 2.4*batt_size[k] for j in range(num_day))
    m.add_constraints(m.sum(eff_c*charge[24*j+i]+eff_c*krd*regdn[24*j+i]  for i in range(24)) <= batt_cap[k] for j in range(num_day))
    m.maximize(m.sum(-lmps[i]*charge[i]+lmps[i]*disch[i]+rd_pr[i]*regdn[i]+ru_pr[i]*regup[i]+sp_pr[i]*spin[i] for i in range(num_hr)))
    m.solve()
    gen_val += [disch[i].solution_value - charge[i].solution_value for i in range(num_hr)]
    rd_val += [regdn[i].solution_value for i in range(num_hr)]
    ru_val += [regup[i].solution_value for i in range(num_hr)]
    sp_val += [spin[i].solution_value for i in range(num_hr)]
    soc_val += [soc[i].solution_value for i in range(num_hr)]
    header += [batt_name[k],"","Profit =",m.objective_value,""]
    del m
    
#Produce Results    
sol_values = pd.DataFrame(data={"Date/Time": multi_batts_data["Date/Time"]})
for k in range(0,num_batt):
    sol_values[gen_h[k]] = gen_val[k*num_hr:(k+1)*num_hr]
    sol_values[regdn_h[k]] = rd_val[k*num_hr:(k+1)*num_hr]
    sol_values[regup_h[k]] = ru_val[k*num_hr:(k+1)*num_hr]
    sol_values[spin_h[k]] = sp_val[k*num_hr:(k+1)*num_hr]
    sol_values[soc_h[k]] = soc_val[k*num_hr:(k+1)*num_hr]
sol_values.columns = pd.MultiIndex.from_tuples(zip(header,sol_values.columns))
sol_values.to_csv("./multi_batt_result.csv", sep=',',index=False)

In [ ]:
#Temporal Data Template
#Run After Running Battery Without Renewables Code Above
import pandas as pd
import numpy as np
import math
opt_result = pd.read_csv('multi_batt_result.csv')
result = opt_result.values
dates = []; date_list = []; prof_name = []; hour_number = {}; generation =[]
for d in range(0,num_day):
    dates += [result[24*d+1,0].split(" ",1)[0]]
for k in range(0,num_batt):
    prof_name += [batt_name[k]]*len(dates)
    date_list += dates
batt_gen = pd.DataFrame(data={"Profile Name": prof_name, "Date": date_list})
for h in range(1,25):
    generation =[]
    hour_number[h-1] = "{0}".format(h)
    for k in range(0,num_batt):
        for d in range(0,num_day):
            generation += [float(result[24*d+h,5*k+1])]
    batt_gen["{0}".format(h)]=generation
batt_gen.to_csv("./batt_gen.csv", sep=',',index=False)

In [ ]:
#Profit Calculation, Battery without Renewables
import pandas as pd
import numpy as np
import math
multi_batts_data = pd.read_csv('Multi_Batts_LP_Data.csv')
batt = multi_batts_data.values
opt_result = pd.read_csv('multi_batt_result.csv')
result = opt_result.values

num_batt = 463; num_hr = len(batt)
batt_name = list(multi_batts_data.columns[7:num_batt+7])
rd_pr = batt[:,4].tolist(); ru_pr = batt[:,5].tolist(); sp_pr = batt[:,6].tolist()
batt_name_list=[]; energy_prof=[0]*num_batt; regdn_income=[0]*num_batt; regup_income=[0]*num_batt; spin_income=[0]*num_batt
for k in range(0,num_batt):
    batt_name_list += [batt_name[k]]
    lmps = batt[:,7+k].tolist()
    gen = result[1:num_hr+1,1+5*k].tolist()
    regdn = result[1:(num_hr+1),(2+5*k)].tolist(); regup = result[1:(num_hr+1),(3+5*k)].tolist(); spin = result[1:(num_hr+1),(4+5*k)].tolist()
    gen = list(map(float,gen)); regdn = list(map(float,regdn)); regup = list(map(float,regup)); spin = list(map(float,spin))
    energy_prof[k] += sum(x*y for x,y in zip(gen,lmps))
    regdn_income[k] += sum(x*y for x,y in zip(regdn,rd_pr))
    regup_income[k] += sum(x*y for x,y in zip(regup,ru_pr)) 
    spin_income[k] += sum(x*y for x,y in zip(spin,sp_pr))
result_profits = pd.DataFrame(data={"Unit Name": batt_name_list, "Energy Revenue": energy_prof, "Reg Down Profit": regdn_income, "Reg Up Profit": regup_income, "Spin Profit": spin_income})
result_profits.to_csv("./multi_batt_result_profits.csv", sep=',',index=False)

In [ ]:
#Profit Calculation w/ renewables
import pandas as pd
import numpy as np
import math
multi_batts_data = pd.read_csv('Multi_Batts_LP_Data.csv')
batt = multi_batts_data.values
opt_result = pd.read_csv('multi_batt_result.csv')
result = opt_result.values

num_batt = 882; num_hr = len(batt)
batt_name = list(multi_batts_data.columns[7:num_batt+7])
rd_pr = batt[:,4].tolist(); ru_pr = batt[:,5].tolist(); sp_pr = batt[:,6].tolist()
batt_name_list=[]; energy_prof=[0]*num_batt; regdn_income=[0]*num_batt; regup_income=[0]*num_batt; spin_income=[0]*num_batt
for k in range(0,num_batt):
    batt_name_list += [batt_name[k]]
    lmps = batt[:,7+k].tolist()
    gen = result[1:(num_hr+1),1+6*k].tolist(); re_charge = result[1:(num_hr+1),2+6*k].tolist()
    regdn = result[1:(num_hr+1),(3+6*k)].tolist(); regup = result[1:(num_hr+1),(4+6*k)].tolist(); spin = result[1:(num_hr+1),(5+6*k)].tolist()
    gen = list(map(float,gen)); re_charge = list(map(float,re_charge))
    regdn = list(map(float,regdn)); regup = list(map(float,regup)); spin = list(map(float,spin))
    tot_gen = [gen[i]-re_charge[i] for i in range(num_hr)]
    energy_prof[k] += sum(x*y for x,y in zip(tot_gen,lmps))
    regdn_income[k] += sum(x*y for x,y in zip(regdn,rd_pr))
    regup_income[k] += sum(x*y for x,y in zip(regup,ru_pr)) 
    spin_income[k] += sum(x*y for x,y in zip(spin,sp_pr))
result_profits = pd.DataFrame(data={"Unit Name": batt_name_list, "Energy Revenue": energy_prof, "Reg Down Profit": regdn_income, "Reg Up Profit": regup_income, "Spin Profit": spin_income})
result_profits.to_csv("./multi_batt_result_profits.csv", sep=',',index=False)

In [ ]:
#Solar Profit Calculation
import pandas as pd
import numpy as np
import math
solar_units_data = pd.read_csv('Renewable_Unit_LMPs.csv')
solar = solar_units_data.values
solar_profiles = pd.read_csv('solar_profiles.csv')
profiles = solar_profiles.values

num_units = 882; num_hr = len(solar)
solar_name = list(solar_units_data.columns[3:num_units+3])
solar_name_list = []; solar_revenue = [0]*num_units
bus_id = solar[0:num_units,1].tolist()
re_size = profiles[0:num_units,1].tolist(); counties = solar[0:num_units,2].tolist()
for k in range(0,num_units):
    solar_name_list += [solar_name[k]]
    lmps = solar[:,3+k].tolist()
    gen = solar_profiles["{0}".format(counties[k])].values
    gen = [i*re_size[k]*0.01 for i in gen]
    solar_revenue[k] += sum(x*y for x,y in zip(gen,lmps))
result_profits = pd.DataFrame(data={"Unit Name": solar_name_list, "Bus ID": bus_id,"Renewable Profit": solar_revenue})
result_profits.to_csv("./result_solar_profits.csv", sep=',',index=False)